Since [OpenPaths](https://openpaths.cc/) is dead, we switched to [WHIB](http://www.bleatinc.com/), which isn't [owned by Facebook](https://moves-app.com/) and doesn't [drain the battery of my phone quickly](http://www.bigpaua.com/arc/).
A 2$ `premium` feature let's you export all the data as CSV, which is easy to parse and display.

In [1]:
import glob
import pandas
import geopy
from geopy.geocoders import Nominatim  # Address search from OpenStreetMap
import matplotlib.pylab as plt
import folium
import folium.plugins

In [2]:
whichyear = 2018

In [3]:
plt.rcParams['figure.figsize'] = (16, 9)  # We live in a widescreen world

In [4]:
locations = pandas.read_csv(sorted(glob.glob('journey*.csv'))[-1])

In [5]:
locations.drop(['Crumb'], axis =1, inplace=True)

In [6]:
locations.head()

,LocalDate,LocalTime,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
0,03.02.18,23:37,46.935401,7.418162,553,65
1,04.02.18,09:01,46.935344,7.417897,553,50
2,04.02.18,09:02,46.932949,7.422317,553,57
3,04.02.18,09:12,46.932855,7.422384,565,38
4,04.02.18,09:14,46.923639,7.415233,565,61


In [7]:
locations.tail()

,LocalDate,LocalTime,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
14388,09.01.19,18:15,46.940836,7.418846,546,65
14389,09.01.19,18:16,46.940993,7.418745,0,79
14390,09.01.19,18:18,46.935581,7.417921,0,95
14391,09.01.19,18:18,46.935573,7.418067,0,0
14392,09.01.19,18:23,46.935289,7.418211,550,165


In [8]:
locations.describe()

,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
count,14393.000000,14393.000000,14393.000000,14393.000000
mean,46.551399,7.647089,534.772181,210.073369
std,1.655602,0.565627,421.566215,2261.191264
min,39.118591,6.630272,-143.000000,0.000000
25%,46.932575,7.418067,492.000000,43.000000
50%,46.938937,7.428877,547.000000,65.000000
75%,46.948962,7.451904,556.000000,82.000000
max,49.479696,9.788838,2724.000000,149000.000000


In [9]:
geopy.geocoders.options.default_user_agent = 'Jahresrückblick Habi'

In [10]:
location_average = [locations['Latitude'].mean(), locations['Longitude'].mean()]
location_north = [locations[locations.Latitude == locations.Latitude.max()].Latitude.values[0],
                             locations[locations.Latitude == locations.Latitude.max()].Longitude.values[0]]
location_east = [locations[locations.Longitude == locations.Longitude.max()].Latitude.values[0],
                            locations[locations.Longitude == locations.Longitude.max()].Longitude.values[0]]
location_south = [locations[locations.Latitude == locations.Latitude.min()].Latitude.values[0],
                             locations[locations.Latitude == locations.Latitude.min()].Longitude.values[0]]
location_west = [locations[locations.Longitude == locations.Longitude.min()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.min()].Longitude.values[0]]

In [11]:
debug = False

In [12]:
# Address lookup with `geopy` and https://wiki.openstreetmap.org/wiki/Nominatim
geolocator = Nominatim()
location_average_geo = geolocator.reverse(location_average)
if debug:
    print(location_average_geo.raw)
name_average = location_average_geo.raw.get('address').get('village')
print('The average location in %s was in %s' % (whichyear, name_average))

The average location in 2018 was in Kandergrund


In [13]:
location_north_geo = geolocator.reverse(location_north)
if debug:
    print(location_north_geo.raw)
name_north = location_north_geo.raw.get('address').get('city')
print('The northmost location in %s was in %s' % (whichyear, name_north))

The northmost location in 2018 was in Mannheim


In [14]:
location_west_geo = geolocator.reverse(location_west)
if debug:
    print(location_west_geo.raw)
name_west = location_west_geo.raw.get('address').get('town')
print('The southmost location in %s was in %s' % (whichyear, name_west))

The southmost location in 2018 was in Yverdon


In [15]:
location_south_geo = geolocator.reverse(location_south)
if debug:
    print(location_south_geo.raw)
name_south = location_south_geo.raw.get('address').get('village')
print('The southmost location in %s was in %s' % (whichyear, name_south))

The southmost location in 2018 was in Crabonaxa/Villasimius


In [16]:
location_east_geo = geolocator.reverse(location_east)
if debug:
    print(location_east_geo.raw)
name_east = location_east_geo.raw.get('address').get('town')
print('The southmost location in %s was in %s' % (whichyear, name_east))

The southmost location in 2018 was in Thiniscole/Siniscola


In [17]:
tileprovider = 'Cartodb Positron'
# tileprovider = 'Stamen Toner'

In [18]:
zoom_start = 5

In [19]:
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
# Extreme locations
radius = 10
folium.CircleMarker(location=location_average,
                    radius=radius,
                    popup='Average location in %s' % name_average,
                   ).add_to(m)
folium.CircleMarker(location=location_north,
                    radius=radius,
                    popup='Northmost location in %s' % name_north,
                   ).add_to(m)
folium.CircleMarker(location=location_east,
                    radius=radius,
                    popup='Northmost location in %s' % name_east,
                   ).add_to(m)
folium.CircleMarker(location=location_south,
                    radius=radius,
                    popup='Northmost location in %s' % name_south,
                   ).add_to(m)
folium.CircleMarker(location=location_west,
                    radius=radius,
                    popup='Northmost location in %s' % name_west,
                   ).add_to(m)
# All locations, in different ways
singlepoints = False
fast = True
if singlepoints:
    for c, loc in locations.iterrows():
        # not everyone, but every fifth one
#        if not c % 5:
            folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                radius=2,
                                popup='%s@%s' % (loc.LocalDate, loc.LocalTime)
                               ).add_to(m)
else:
    if fast:
        # FastMarkerCluster
        m.add_child(folium.plugins.FastMarkerCluster(locations[['Latitude', 'Longitude']].values.tolist()))
    else:
        # Markercluster
        mc = folium.plugins.MarkerCluster()
        for c, loc in locations.iterrows():
            mc.add_child(folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                             popup='%s@%s' % (loc.LocalDate, loc.LocalTime)))
        m.add_child(mc)
m.save('map-points.html')
m

In [20]:
# Viridis colormap from here: https://www.thedataschool.co.uk/gwilym-lockwood/viridis-colours-tableau/
gradient={0.00: '#440154FF',
          0.05: '#481567FF', 
          0.10: '#482677FF', 
          0.15: '#453781FF', 
          0.20: '#404788FF', 
          0.25: '#39568CFF', 
          0.30: '#33638DFF', 
          0.35: '#2D708EFF', 
          0.40: '#287D8EFF', 
          0.45: '#238A8DFF', 
          0.50: '#1F968BFF', 
          0.55: '#20A387FF', 
          0.60: '#29AF7FFF', 
          0.65: '#3CBB75FF', 
          0.70: '#55C667FF', 
          0.75: '#73D055FF', 
          0.80: '#95D840FF', 
          0.85: '#B8DE29FF', 
          0.90: '#DCE319FF', 
          0.95: '#FDE725FF'}

In [21]:
# Show a heatmap instead of single points
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
# Extreme locations
folium.CircleMarker(location=location_average,
                    radius=radius,
                    popup='Average location in %s' % name_average,
                   ).add_to(m)
folium.CircleMarker(location=location_north,
                    radius=radius,
                    popup='Northmost location in %s' % name_north,
                   ).add_to(m)
folium.CircleMarker(location=location_east,
                    radius=radius,
                    popup='Most eastern location in %s' % name_east,
                   ).add_to(m)
folium.CircleMarker(location=location_south,
                    radius=radius,
                    popup='Southmost location in %s' % name_south,
                   ).add_to(m)
folium.CircleMarker(location=location_west,
                    radius=radius,
                    popup='Most western location in %s' % name_west,
                   ).add_to(m)
# Add heatmap
folium.plugins.HeatMap(locations[['Latitude', 'Longitude']].values.tolist(),
                       gradient=gradient, blur=0).add_to(m)
m.save('map-heat.html')
m